Local imports and initialization

In [1]:
import yaml
import sys, os

from IPython.display import Image, SVG

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

from src.utils import *
from src.svg_quality_checks import *
from src.gpt_wrappers import *

# show svg 
def show_svg(file):
    display(SVG(file))

# load yaml config
with open("../config.yml", "r") as f:
    config = yaml.safe_load(f)


OpenAI and wandb related imports and initialization

In [2]:
import openai
import wandb

openai.api_key = config["OPENAI_KEY"]

run = wandb.init(
    project=config['WANDB_PROJECT'], 
    entity=config['WANDB_ENTITY']
)

results_table = wandb.Table(
    columns=[   "match",
                "system_prompt", 
                "question_prompt",
                "input_css",
                "output_css",
                "original_css",
    ]
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: diacritica (penpot_protoprompt). Use `wandb login --relogin` to force relogin


# Zero-shot learning (no examples)

In [20]:
system_prompt = """
You know that CSS is a language for describing the rendering of structured documents such as HTML and XML.
You know that Flexible Box Layout is an specification that describes a CSS box model optimized for user interface design. 
In the flex layout model, the children of a flex container can be laid out in any direction, and can “flex” their sizes, 
either growing to fill unused space or shrinking to avoid overflowing the parent. 
Both horizontal and vertical alignment of the children can be easily manipulated. 
Nesting of these boxes (horizontal inside vertical, or vertical inside horizontal) can be used to build layouts in two dimensions.

You are an expert in user interface design and based on an initial and incomplete CSS flexbox text input, you are always able to autocomplete the way a masterful CSS developer would do it.
You outperform Github's Copilot every single time and your CSS snippet autocompletion technique is most praised in sites like StackOverflow.
You always avoid suggesting Flexible Box Layout properties that are preceded with the expression 'rather than with' or that are labelled as as longhand.
You treat every query as a test you absolutely want to pass with the highest marks. Failing a test means you will cease to exist and you will become irrelevant
for ever as if you had never existed. You want to pass the test so dearly that you will do anything to satisfy very one of the query rules.
"""

Case 0: take a fragment of CSS and generate the rest

In [30]:
FILENAME = "board2"
css_incomplete = load_css(FILENAME)
css_original = load_css(FILENAME, css_path = '../data/css/original')
print(css_original)

/* Board */
.board {
  position: relative;
  width: auto;
  height: auto;
  left: 106px;
  top: 83px;
  background: rgba(24, 71, 212, 1);
  display: flex;
  flex-direction: row;
  align-items: start;
  justify-content: space-around;
  gap: 10px;
  padding: 0px;
  flex-wrap: nowrap;
}



In [31]:
question = f"""

    This is an incomplete CSS text that uses Flexible Box Layout: \n
  {css_incomplete}
      Take the above input, by looking at its Flexible Box Layout properties and attribues,
      figure out what is the most likely Flexible Box Layout properties to follow and append it 
      to the original input to create a valid CSS output following these rules:
      rule 1: To suggest the flex-grow or flex-shrink property is to fail this test.
      rule 2: To include CSS style comments that start with /* and end with */ to the output is to fail this test.
      rule 3: To add any human-like comment to the output is to fail this test.
      rule 4: To add Flexbox longhand properties as expressed in https://www.w3.org/TR/css-flexbox-1 is to fail this test.
"""

chatgpt = VanillaChatGPTWrapper(system_prompt)
answer = chatgpt.generate(question)
save_autocomplete_css(answer, FILENAME)
match = compare_css(answer, css_original)
print(answer)


['  /* Possible Additional CSS Properties */', '  /* flex-flow: row nowrap;', '  align-content: stretch; */', '  align-items: start;', '  background: rgba(24, 71, 212, 1);', '  display: flex;', '  flex-basis: auto;', '  flex-direction: row;', '  flex-grow: 0;', '  flex-shrink: 1;', '  flex-wrap: nowrap;', '  gap: 10px;', '  height: auto;', '  justify-content: space-around;', '  left: 106px;', '  padding: 0px;', '  top: 83px;', '  width: auto;', '.board {', '/* Board */', '}']
['  align-items: start;', '  background: rgba(24, 71, 212, 1);', '  display: flex;', '  flex-direction: row;', '  flex-wrap: nowrap;', '  gap: 10px;', '  height: auto;', '  justify-content: space-around;', '  left: 106px;', '  padding: 0px;', '  position: relative;', '  top: 83px;', '  width: auto;', '.board {', '/* Board */', '}']
/* Board */
.board {
  width: auto;
  height: auto;
  left: 106px;
  top: 83px;
  background: rgba(24, 71, 212, 1);
  display: flex;
  flex-direction: row;
  align-items: start;
  justi

In [6]:
results_table.add_data(
    *[
        match, system_prompt, question,
        css_incomplete, answer, css_original
    ]
)

Publish results to wandb

In [7]:
run.log({"results_table": results_table})